# Predicting Google stock prices Using RNN GRU Model

This use-case provides a demo of how RNN and GRU can be used to model time series data. In our use-case, we will use 'Google stock price' data to predict the number of future stock prices based on the available historical data. 

Workflow:


1.   Understanding the problem
2.   Collecting the data
3.   Data preprocessing
4.   Build Model using RNN & Grated Recurrent Unit (GRU)
5.   Training the dataset
6.   Predict 


## 1. Understanding the problem
You have been given a dataset that contains google stock prices. We have to model the time series data using RNN GRU sequential model.

Import python library

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional,SimpleRNN
from tensorflow.keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [2]:
# Some functions to help out with predictions

def rmse_return(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

## 2. Collecting the Data
The dataset trainset.csv will be used for the demo. The dataset contains google stock prices. The features are:
- "Date" -  The date on which the data was collected.
- "open" - The opening price for the specified date(s).

- "close" - The closing price for the specified date(s).

- "high" - The high price for the specified date(s).

- "low" - The low price for the specified date(s).

- "volume" - The volume for the specified date(s).
 

Load the dataset using pandas read_csv function.

In [3]:
gsheet_url="https://docs.google.com/spreadsheets/d/1Sqf4OdzWkm_BkfnSw2YAMF_J7vEApKu3uJosyqNgPrE/edit#gid=0"

In [4]:
url_1 = gsheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [5]:
# First, we get the data
dataset = pd.read_csv(url_1)
dataset.head()

,Date,Open,High,Low,Close,Volume
0,6/21/2016 16:00:00,698.40,702.77,692.01,695.94,1465634
1,6/22/2016 16:00:00,699.06,700.86,693.08,697.46,1184318
2,6/23/2016 16:00:00,697.45,701.95,687.00,701.87,2171415
3,6/24/2016 16:00:00,675.17,689.40,673.45,675.22,4449022
4,6/27/2016 16:00:00,671.00,672.30,663.28,668.26,2641085


In [6]:
# Converting dates:
dataset.Date = pd.to_datetime(dataset.Date)

In [7]:
dataset.Date=[date.strftime(format="%Y/%m/%d") for date in dataset.Date]

In [8]:
dataset.Date = pd.to_datetime(dataset.Date)

In [9]:
dataset=dataset.set_index('Date')

In [10]:
dataset.head()

,Open,High,Low,Close,Volume
Date,,,,,
2016-06-21,698.40,702.77,692.01,695.94,1465634
2016-06-22,699.06,700.86,693.08,697.46,1184318
2016-06-23,697.45,701.95,687.00,701.87,2171415
2016-06-24,675.17,689.40,673.45,675.22,4449022
2016-06-27,671.00,672.30,663.28,668.26,2641085


In [11]:
# Checking for missing values in Training and Test set
training_set = dataset[dataset.index.year<2020].High
test_set = dataset[dataset.index.year>=2020].High

In [12]:
print("The dataset has information from" , dataset.index.min().strftime("%d/%m/%Y"),"to date",dataset.index.max().strftime("%d/%m/%Y"))

The dataset has information from 21/06/2016 to date 10/12/2021


## Exploratory Data Analysis

In [13]:
import plotly.express as px
import plotly.graph_objects as go


In [14]:
dataset.head()

,Open,High,Low,Close,Volume
Date,,,,,
2016-06-21,698.40,702.77,692.01,695.94,1465634
2016-06-22,699.06,700.86,693.08,697.46,1184318
2016-06-23,697.45,701.95,687.00,701.87,2171415
2016-06-24,675.17,689.40,673.45,675.22,4449022
2016-06-27,671.00,672.30,663.28,668.26,2641085


In [15]:
# Yearly standard deviation
dataset.resample('y',axis=0).agg('std')

,Open,High,Low,Close,Volume
Date,,,,,
2016-12-31,31.091369,30.916665,30.916993,30.317360,737157.690341
2017-12-31,77.249509,78.340330,76.357837,77.232928,638257.185161
2018-12-31,67.218290,64.329414,69.119072,67.315551,716745.535941
2019-12-31,81.571801,81.109499,82.826251,81.765380,621731.444100
2020-12-31,176.792950,174.859537,178.934771,175.967843,763314.693682
2021-12-31,366.532090,364.868973,364.284404,364.266982,498402.797368


#### Yearly Candlestick

In [16]:
temp=dataset.copy()
temp['Date']=dataset.index

In [17]:
fig = go.Figure()

for year in range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1):
    sample = temp[temp.Date.dt.year==year]
    fig.add_traces (go.Candlestick(x=sample.index,
                open=sample.Open,
                high=sample.High,
                low=sample.Low,
                close=sample.Close,
                visible=True,showlegend=False
                                  )
                   )
# Create menu items
items=[]
plo='Candlestick plot for '
all_plot=[True for i in range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1)]
items.append(dict(label = 'All',
                    method = 'update',
                    args = [{'visible':all_plot },
                            {'title': plo+'complete data',
                            'showlegend':False}]))
ind=-1
for year in range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1):
    visible_plot=[False for i in range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1)]
    ind+=1
    visible_plot[ind]=True
    items.append(
            dict(label = str(year),
                    method = 'update',
                    args = [{'visible':visible_plot },
                            {'title': plo+str(year),
                            'showlegend':False}])
                )
fig.update_layout(dragmode=False,showlegend=False,template='plotly_dark',
        updatemenus=[go.layout.Updatemenu(
            active=0,
            buttons=items,
                pad={"r": 10, "t": 2},
                showactive=True,
                x=0,
                xanchor="right",
                y=1.15,
                yanchor="top"
            )
        ]
        )

fig.show()

### Area Chart

In [18]:
fig = go.Figure()
years=len(range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1))

for year in range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1):
    sample = temp[temp.Date.dt.year==year]
    fig.add_traces( go.Scatter(name=year,x=sample.index, y=sample.Open, fill='tozeroy',visible=True,showlegend=False)
                
            )
            

# Create menu items
items=[]
plo='Area plot for '
all_plot=[True for i in range(years)]
items.append(dict(label = 'All',
                    method = 'update',
                    args = [{'visible':all_plot },
                            {'title': plo+'complete data',
                            'showlegend':False}]))
ind=-3
for year in range(int(dataset.index.min().strftime("%Y")),int(dataset.index.max().strftime("%Y"))+1):
    visible_plot=[False for i in range(years)]
    ind+=1
    visible_plot[ind]=True
    items.append(
            dict(label = str(year),
                    method = 'update',
                    args = [{'visible':visible_plot },
                            {'title': plo+str(year),
                            'showlegend':False}])
                )
fig.update_layout(dragmode=False,showlegend=False,template='plotly',
        updatemenus=[go.layout.Updatemenu(
            active=0,
            buttons=items,
                pad={"r": 10, "t": 2},
                showactive=True,
                x=0,
                xanchor="right",
                y=1.13,
                yanchor="top"
            )
        ]
        )

fig.show()

In [19]:
# We have chosen 'High' attribute for prices. Let's see what it looks like
fig=go.Figure()
fig.add_traces( go.Scatter(x=dataset[dataset.index.year<2020].index, y=dataset[dataset.index.year<2020].Open, 
                           fill='tozeroy',
                           visible=True,
                           marker={'color':'#ff006e'},
                           showlegend=False,
                           name='Training set (Before 2020)'
                          )
                        
            )
fig.add_traces( go.Scatter(x=dataset[dataset.index.year>=2020].index, y=dataset[dataset.index.year>=2020].Open, 
                           fill='tozeroy',
                           visible=True,
                           marker={'color':'#3a86ff'},
                           showlegend=False,
                           name='Testing set (After 2020)'
                          )
            )
fig.update_layout(title='Google Stock Price Training and testing set distribution',dragmode=False)
fig.show()

In [20]:
len(training_set),len(test_set)

(889, 491)

In [21]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(np.array(training_set).reshape(-1,1))

In [22]:
# Data Preparation
def picker(series,pos,days):
    return list(series[pos:pos+days]), series[pos+days],np.mean(np.array(series[pos:pos+days]))

def prepare_data(series,days):
    pos=0
    X,y=[],[]
    while pos!=len(series)-days:
        X_entry,y_entry,mean=picker(series,pos,days)
        X_entry= np.array(X_entry)
        X.append(X_entry);y.append(y_entry)

        pos+=1
    return np.array(X),np.array(y)

In [23]:
# Since RNN/GRU/LSTM store long term memory state, we create a data structure with n_days timesteps and 1 output
n_days=60
X_train,y_train=prepare_data(training_set_scaled,n_days)

In [24]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [35]:
X_train

array([[[0.0439873 ],
        [0.04122997],
        [0.04280352],
        ...,
        [0.14146095],
        [0.13558539],
        [0.13769309]],

       [[0.04122997],
        [0.04280352],
        [0.02468601],
        ...,
        [0.13558539],
        [0.13769309],
        [0.14652808]],

       [[0.04280352],
        [0.02468601],
        [0.        ],
        ...,
        [0.13769309],
        [0.14652808],
        [0.14068139]],

       ...,

       [[0.75745633],
        [0.74600837],
        [0.77831673],
        ...,
        [0.99249314],
        [0.97872095],
        [0.99470189]],

       [[0.74600837],
        [0.77831673],
        [0.78807565],
        ...,
        [0.97872095],
        [0.99470189],
        [0.9993215 ]],

       [[0.77831673],
        [0.78807565],
        [0.77057889],
        ...,
        [0.99470189],
        [0.9993215 ],
        [0.98267648]]])

In [25]:
# Now to get the test set ready in a similar way as the training set.
# The following has been done for the 60 entires of test set have 60 previous values which is impossible to get unless we take the whole 
# 'High' attribute data for processing
dataset_total = dataset.copy()
inputs = dataset_total[len(dataset_total)-len(test_set) - 30:].values
inputs = inputs.reshape(-1,1)
inputs  = sc.transform(inputs)

## Gated Recurrent Units

In simple words, the GRU unit does not have to use a memory unit to control the flow of information like the LSTM unit. It can directly makes use of the all hidden states without any control. GRUs have fewer parameters and thus may train a bit faster or need less data to generalize. But, with large data, the LSTMs with higher expressiveness may lead to better results.

They are almost similar to LSTMs except that they have two gates: reset gate and update gate. Reset gate determines how to combine new input to previous memory and update gate determines how much of the previous state to keep. Update gate in GRU is what input gate and forget gate were in LSTM. We don't have the second non linearity in GRU before calculating the outpu, .neither they have the output gate.

## Create the model

In [26]:
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=50, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=1))

In [27]:
regressorGRU.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 60, 50)            2600      
                                                                 
 dropout (Dropout)           (None, 60, 50)            0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 60, 50)            5050      
                                                                 
 dropout_1 (Dropout)         (None, 60, 50)            0         
                                                                 
 gru (GRU)                   (None, 60, 50)            15300     
                                                                 
 dropout_2 (Dropout)         (None, 60, 50)            0         
                                                                 
 gru_1 (GRU)                 (None, 50)                1

## Train the model

In [28]:
# Compiling the GRU
regressorGRU.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
# Fitting to the training set
regressorGRU.fit(X_train,y_train,epochs=50,batch_size=150)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



6/6 [==============================] - 7s 168ms/step - loss: 0.2423
Epoch 2/50
6/6 [==============================] - 1s 166ms/step - loss: 0.0899
Epoch 3/50
6/6 [==============================] - 1s 189ms/step - loss: 0.0461
Epoch 4/50
6/6 [==============================] - 1s 185ms/step - loss: 0.0319
Epoch 5/50
6/6 [==============================] - 1s 180ms/step - loss: 0.0258
Epoch 6/50
6/6 [==============================] - 1s 163ms/step - loss: 0.0169
Epoch 7/50
6/6 [==============================] - 1s 161ms/step - loss: 0.0140
Epoch 8/50
6/6 [==============================] - 1s 171ms/step - loss: 0.0127
Epoch 9/50
6/6 [==============================] - 1s 178ms/step - loss: 0.0115
Epoch 10/50
6/6 [==============================] - 1s 178ms/step - loss: 0.0107
Epoch 11/50
6/6 [==============================] - 1s 164ms/step - loss: 0.0097
Epoch 12/50
6/6 [==============================] - 1s 170ms/step - loss: 0.0092
Epoch 13/50
6/6 [==============================] - 1s 163ms/

The current version version uses a dense GRU network with 100 units as opposed to the GRU network with 50 units in previous version

In [29]:
# The LSTM architecture
lstm = Sequential()
lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
lstm.add(Dropout(0.2))
lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=50, activation='relu'))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1))

In [30]:
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                                 
 dropout_4 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_5 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_6 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 50)               

In [31]:
# Compiling the LSTM
lstm.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
# Fitting to the training set
lstm.fit(X_train,y_train,epochs=50,batch_size=150)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



6/6 [==============================] - 8s 244ms/step - loss: 0.2593
Epoch 2/50
6/6 [==============================] - 2s 263ms/step - loss: 0.0625
Epoch 3/50
6/6 [==============================] - 2s 253ms/step - loss: 0.0861
Epoch 4/50
6/6 [==============================] - 2s 261ms/step - loss: 0.0511
Epoch 5/50
6/6 [==============================] - 2s 258ms/step - loss: 0.0533
Epoch 6/50
6/6 [==============================] - 1s 248ms/step - loss: 0.0470
Epoch 7/50
6/6 [==============================] - 2s 259ms/step - loss: 0.0436
Epoch 8/50
6/6 [==============================] - 2s 265ms/step - loss: 0.0418
Epoch 9/50
6/6 [==============================] - 2s 257ms/step - loss: 0.0392
Epoch 10/50
6/6 [==============================] - 2s 264ms/step - loss: 0.0371
Epoch 11/50
6/6 [==============================] - 2s 260ms/step - loss: 0.0353
Epoch 12/50
6/6 [==============================] - 2s 262ms/step - loss: 0.0308
Epoch 13/50
6/6 [==============================] - 2s 259ms/

## Test the model

In [32]:
X_test,y_test=prepare_data(sc.transform(np.array(training_set[-n_days:].append(test_set)).reshape(-1,1)),n_days)

In [33]:
X,y=prepare_data(sc.transform(np.array(training_set.append(test_set[:n_days])).reshape(-1,1)),n_days)

In [37]:
# Over training set
#what are we doing?

LSTM_predicted_stock_price_train1 = lstm.predict(X.reshape(X.shape[0],X.shape[1],1))
LSTM_predicted_stock_price_train = sc.inverse_transform(LSTM_predicted_stock_price_train1)

In [39]:
LSTM_predicted_stock_price_train

array([[ 814.42596],
       [ 814.5299 ],
       [ 814.6017 ],
       [ 814.64465],
       [ 814.66473],
       [ 814.6682 ],
       [ 814.6631 ],
       [ 814.66174],
       [ 814.67316],
       [ 814.7036 ],
       [ 814.7579 ],
       [ 814.8364 ],
       [ 814.93854],
       [ 815.05865],
       [ 815.1923 ],
       [ 815.33527],
       [ 815.48346],
       [ 815.63306],
       [ 815.78357],
       [ 815.9377 ],
       [ 816.09937],
       [ 816.2669 ],
       [ 816.43915],
       [ 816.6143 ],
       [ 816.7978 ],
       [ 817.00226],
       [ 817.2454 ],
       [ 817.53064],
       [ 817.86334],
       [ 818.2676 ],
       [ 818.74493],
       [ 819.285  ],
       [ 819.88135],
       [ 820.5159 ],
       [ 821.16425],
       [ 821.7939 ],
       [ 822.36884],
       [ 822.85565],
       [ 823.2405 ],
       [ 823.5255 ],
       [ 823.7221 ],
       [ 823.8423 ],
       [ 823.8797 ],
       [ 823.83246],
       [ 823.6963 ],
       [ 823.46674],
       [ 823.14984],
       [ 822.

In [41]:
# Over training set
GRU_predicted_stock_price_train = regressorGRU.predict(X.reshape(X.shape[0],X.shape[1],1))
GRU_predicted_stock_price_trainb = sc.inverse_transform(GRU_predicted_stock_price_train)

In [42]:
# Over test set
LSTM_predicted_stock_price = lstm.predict(X_test.reshape(X_test.shape[0],X_test.shape[1],1))
LSTM_predicted_stock_price_test = sc.inverse_transform(LSTM_predicted_stock_price)

In [43]:
# Over test set
GRU_predicted_stock_price = regressorGRU.predict(X_test.reshape(X_test.shape[0],X_test.shape[1],1))
GRU_predicted_stock_price_test = sc.inverse_transform(GRU_predicted_stock_price)

In [44]:
# We have chosen 'High' attribute for prices. Let's see what it looks like
fig=go.Figure()
fig.add_traces( go.Scatter(x=dataset[dataset.index.year<2020].index, y=dataset[dataset.index.year<2020].Open, 

                           visible=True,
                           marker={'color':'#ff006e'},
                           showlegend=True,
                           name='Training set (Before 2020)'
                          )
                        
            )
fig.add_traces( go.Scatter(x=dataset[dataset.index.year>=2020].index, y=dataset[dataset.index.year>=2020].Open, 

                           visible=True,
                           marker={'color':'#3a86ff'},
                           showlegend=True,
                           name='Testing set (After 2020)'
                          )
              )

fig.add_traces( go.Scatter(x=dataset[dataset.index.year>=2020].index, y=[i[0] for i in LSTM_predicted_stock_price_test],

                           visible=True,
                           marker={'color':'#007f5f'},
                           showlegend=True,
                           name='Predicted'
                          )
            )

fig.add_traces( go.Scatter(x=dataset[dataset.index.year<2020].index, y=[i[0] for i in LSTM_predicted_stock_price_train],

                           visible=True,
                           marker={'color':'#007f5f'},
                           showlegend=False,
                           name='Predicted'
                          )
            )

fig.add_traces( go.Scatter(x=dataset[dataset.index.year>=2020].index, y=[i[0] for i in GRU_predicted_stock_price_test],
                            
                           visible=True,
                           marker={'color':'#fbc4ab'},
                           showlegend=True,
                           name='Predicted'
                          )
            )

fig.add_traces( go.Scatter(x=dataset[dataset.index.year<2020].index, y=[i[0] for i in GRU_predicted_stock_price_train],

                           visible=True,
                           marker={'color':'#fbc4ab'},
                           showlegend=False,
                           name='Predicted'
                          )
            )
fig.update_layout(title='Google Stock Price Prediction LSTM',dragmode=False)
fig.show()

In [45]:
# Evaluating GRU
rmse_return(test_set,GRU_predicted_stock_price_test)

The root mean squared error is 821.0767277413185.


In [46]:
# Evaluating GRU
rmse_return(test_set,LSTM_predicted_stock_price_test)

The root mean squared error is 789.5838800382503.


As we can see that GRU is performing better than LSTM.